In [1]:
pip install transformers datasets evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.5 MB/s eta 0:00:00


In [2]:
pip install --upgrade datasets fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 23.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load dataset
dataset = load_dataset("wikitext", "wikitext-2-v1")
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# Tokenize function
def tokenize_function(example):
    return tokenizer(example["text"])

# Apply tokenizer
tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/685k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.07M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/618k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [2]:
block_size = 256

def group_texts(examples):
    concatenated = sum(examples["input_ids"], [])
    total_length = (len(concatenated) // block_size) * block_size
    result = {
        "input_ids": [concatenated[i:i + block_size] for i in range(0, total_length, block_size)],
        "attention_mask": [ [1]*block_size for _ in range(0, total_length, block_size) ]
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)


Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [3]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

# Load pretrained GPT-2
model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define training arguments (compatible version)
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    logging_dir="./logs",
    logging_steps=50,  # Logs every 50 steps
    save_strategy="no",  # Avoid save-related errors on older versions
    report_to="none"     # Avoids wandb issues in old environments
)

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
)


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [4]:
# Train
trainer.train()

# Evaluate
eval_results = trainer.evaluate()
import math
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.735800
100,3.541400
150,3.441100
200,3.388400
250,3.381900
300,3.379400
350,3.342900
400,3.327100
450,3.321300
500,3.234700


Perplexity: 17.09


In [5]:
import torch

def top_k_accuracy(logits, labels, k=5):
    topk = torch.topk(logits, k, dim=-1).indices
    correct = topk.eq(labels.unsqueeze(-1)).any(dim=-1)
    return correct.float().mean().item()

model.eval()
dataloader = trainer.get_eval_dataloader()

total, count = 0.0, 0
for batch in dataloader:
    input_ids = batch["input_ids"].to(model.device)
    labels = batch["labels"].to(model.device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids)
        logits = outputs.logits

    # Shift logits and labels to match next-word prediction
    logits = logits[:, :-1, :].contiguous()
    labels = labels[:, 1:].contiguous()

    # Flatten
    logits = logits.view(-1, logits.size(-1))
    labels = labels.view(-1)

    acc = top_k_accuracy(logits, labels, k=5)
    total += acc
    count += 1

print(f"Top-5 Accuracy: {total / count:.2f}")


Top-5 Accuracy: 0.67
